<a href="https://colab.research.google.com/github/salarMokhtariL/House-Prices/blob/main/Houes_Price_Predictition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Houes Price Predictition

> By Salar Mokhtari *Laleh*

# Import the necessary libraries

In [2]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [3]:
train_data = pd.read_csv('https://raw.githubusercontent.com/salarMokhtariL/House-Prices/main/Dataset/train.csv')
test_data = pd.read_csv('https://raw.githubusercontent.com/salarMokhtariL/House-Prices/main/Dataset/test.csv')

# Preprocess the data
train_data = train_data.select_dtypes(include=np.number)
test_data = test_data.select_dtypes(include=np.number)

# Separate the target variable (SalePrice) from the training data
y_train = train_data['SalePrice']
X_train = train_data.drop('SalePrice', axis=1)

# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [4]:
class HousePriceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.float32)
        
    def __getitem__(self, index):
        return self.X[index], self.y[index]
    
    def __len__(self):
        return len(self.X)

# Define the data loaders
train_dataset = HousePriceDataset(X_train, y_train)
val_dataset = HousePriceDataset(X_val, y_val)
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [5]:
class HousePriceModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Define the model
input_size = X_train.shape[1]
hidden_size = 64
output_size = 1
model = HousePriceModel(input_size, hidden_size, output_size)


In [6]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
# Train the model
epochs = 100
for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1} - Training loss: {running_loss/len(train_loader):.4f}")